# Rezultate obtinute

## Scurta descriere

Arhitectura este una de tip client-server, in care serverul realizeaza in mare parte agregarea de modele si mentine date despre clustere (numarul de clustere, componenta fiecarui cluster). Fiecare cluster va contine mai multi clienti, care au caracteristici similare si care vor participa in invatarea unui model per cluster, intr-o maniera similara cu cea din **FedAvg** (federated learning, in care agregarea se face realizand media ponderata a modelelor). Adaugarea unui nou client intr-un cluster se face printr-o coada "de asteptare" (wait queue), in care clientii vor fi adaugati dupa procesul de agregare (daca se adauga clienti in mod dinamic), sau inainte de a incepe orice operatie (cu clientii initiali).

Prin aceasta arhitectura nu se face niciun schimb de date, ci doar de modele antrenate. Prin urmare, complexitatea realizarii comunicatiei se va raporta la numarul de modele trimise prin retea (care, in acest caz, creste in mod liniar cu numarul de runde). 

Pentru a creste securitatea si a scadea numarul de calcule efectuate, se foloseste un **denoise autoencoder** care invata o reprezentare a datelor pe mai putine dimensiuni (in cazul liniar, se comporta aproape la fel ca PCA - Principal Component Analysis; dar autoencoder-ul poate sa realizeze reducerea dimensiunilor si pe caz neliniar [1]).

Serverul va retine atat un model global (care este antrenat pe datele publice, si care poate fi actualizat in procesul de invatare din fiecare cluster) si mai multe modele asociate fiecarui cluster. Predictia se poate realiza prin mai multe metode:
* in cazul in care datele sunt in cadrul unui client dintr-un cluster, se aplica modelul invatat la client
* in cazul in care datele sunt din afara "cloud-ului", se poate utiliza:
    * modelul global
    * toate modelele asociate clusterelor, si apoi sa se foloseasca un mecanism de votare/valoare maxima/valoare medie/etc

## Initializare

In procesul de initializare, se creaza un model global si apoi, pe baza acestui model, se creaza mai multe clustere. Algoritm:
1. Serverul calculeaza un model generic pe date disponibil public
2. Cand o parte dintre clienti se conecteaza la server, fiecare o sa primeasca modelul antrenat pe server
3. Fiecare client antreneaza modelul cu datele pe care le detine
4. Dupa antrenare, fiecare client trimite modelul $ m_i $ antrenat la server
5. Serverul compara modelele si creaza clustere astfel (pseudocod):

In [ ]:
for client in all_clients:
    if len(clusters) == 0: # There are no clusters created
        new_cluster = create_cluster(client) # Create a new cluster having only "client"
        clusters.append(new_cluster)
    else:
        for c in clusters:
            if c[model] is similar to m:
                add_to_queue(c, client) # Add the client to the queue of cluster "c"
                break
        if no similar model found:
            new_cluster = create_cluster(client) # Create a new cluster having only "client"
            clusters.append(new_cluster)        

## Wait queue
Wait queue este folosit pentru a adauga noi clienti in cadrul unui cluster, in mod dinamic. In timpul procesului de invatare, se adauga clientii noi in coada asociata clusterului, iar dupa ce se termina runda de invatare, clientii sunt extrasi din coada si introdusi in cluster, unde vor participa la urmatoarele runde.

## Compararea similaritatii modelelor
Intrucat nu se doreste partajarea datelor pentru a determina faptul ca mai multi clienti au caracterisici similare (de exemplu, folosind K-Means Clustering), se considera modelul ca un black-box si daca doi clienti au modele similare, atunci acestea ar trebui sa produca iesiri similare pe acelasi set de date. Important de notat ca datele clientilor, in general, sunt non-IID (non independent and identically distributed), deci functia invatata ar trebui sa produca iesiri biased.
Formula folosinta pentru a alege clusterul din care face parte clientul, pe baza modelului invata si a modelelor existente deja pentru clustere, este:
$$ c = argmin_{c \in C} \frac{1}{n} \sum_{i=1}^n ||f(x_i) - g_c(x_i)||_2 $$
unde,  
$c$ - clusterul nou ales  
$C$ - multime tuturor clusterelor  
$n$ - numarul de exemple din setul de date  
$f$ - modelul antrenat de client  
$g_c$ - modelul clusterului $c$  

De asemenea, se aleg doar acele valori care sunt mai mici decat un **threashold**, daca valoarea minima este foartea mare (e.g. 1000), modelele sunt complet diferite. Functia descrisa anterior calculeaza eroarea medie a iesirilor si alege clusterul in care modelul genereaza eroarea cea mai mica.  
**Nota:** Datele $x_i$ trebuie sa fie distribuite IID pentru a putea determina corect clusterul din care face parte clientul.

## Invatarea in cadrul unui cluster
1. Serverul alege un numar aleatoriu de clienti care sa participe la invatare in etapa (epoca) curenta
2. Se trimite modelul agregat catre toti clientii alesi
3. Fiecare client invata (folosint invatare prin transfer) pe datele pe care le are (aici se aplica noise asupra datelor + autoencoder pentru reducerea dimensiunilor)
4. Modelul este partajat folosind MPC intre 2-3 parteneri (daca este posibil) pentru calcularea mediei modelelor (pas optional, se face pentru a creste securitatea)
5. Se trimite modelul la server
6. Severul face agregarea modelelor primite si se reia de la pasul 1.

## Arhitectura (retea neurala) utilizata in testare
* Retea convolutionara, dimensiune kernel = 3
* Activare folosind ReLU
* Max pooling, dimensiune kernel = 2
* Retea convolutionara, dimensiune kernel = 3
* Activare folosind ReLU
* Max pooling, dimensiune kernel = 2
* Fully connected layer
* Activare folosind ReLU
* Fully connected layer
* Activare folosind ReLU
* Fully connected layer
* Activare folosind log softmax

## Arhitectura utilizata pentru autoencoder
* 6 layer-e fully connected
* functia de activare ReLU intre layer-e
* la iesire, am testat cu functia sigmoid (+ normalizare date in (0, 1)) si tanh (setul de date era normalizat in (-1, 1))

## Setul de date
Date de la senzori de telefon (Samsung Galaxy S II), pentru recunoasterea activitatilor:  
http://archive.ics.uci.edu/ml/datasets/Smartphone-Based+Recognition+of+Human+Activities+and+Postural+Transitions


## Testarea
Testarea s-a realizat in mai multe conditii:
* diferite valori pentru parametrii de invatare (rata de invatare, numarul de epoci, batch size)
* diferite arhitecturi de autoencorer (dimensiuni diferite pentru hidden layers) si valori diferite pentru epoci si batch size
* numar diferit de clienti (5 clienti sau 10 clienti) cu seturi de date distribuite fie 1-la-1 (25 pentru server si 5 pentru clienti) fie 2-la-1 (15 pentru server si 10 pentru clieti) (setul de date contine date de la 30 de persoane)
* cu sau fara layer de personalizare (era ultimul layer fully-connected si care nu se modifica)

### Grafice (o parte din rulari)
* 5 clienti, 10 epoci FL, 5 clustere, fara layer de personalizare, fara autoencoder, distributie date 1-la-1
<img src="IMG1.png">

* 5 clienti, 20 epoci FL, 5 clustere, fara layer de personalizare, fara autoencoder, distributie date 2-la-1
<img src="IMG2.png">

* 10 clienti, 20 epoci FL, 7 clustere, fara layer de personalizare, fara autoencoder, distributie date 1-la-1
<img src="IMG3.png">

* 5 clienti, 10 epoci FL, 2 clustere, cu layer de personalizare, cu autoencoder, distributie date 1-la-1
<img src="IMG4.png">

## Referinta
[1] https://stats.stackexchange.com/a/120096